<a href="https://colab.research.google.com/github/Pierre-Faure/Cine_ML/blob/main/notebooks/XGB_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Montage disque google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Indique quel GPU est utilisé
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Oct 25 08:13:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!pip install xgboost==1.6.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 255.9 MB 47 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
import pandas as pd
import numpy as np
import os
import sys
from ast import literal_eval
from time import time

PROJECT_PATH = "gdrive/MyDrive/Data Science/Cine ML/"

sys.path.append(PROJECT_PATH)
FILE_PATH = os.path.join(PROJECT_PATH, "data", "complete_df.csv")

In [ ]:
print(FILE_PATH)

gdrive/MyDrive/Data Science/Cine ML/data/complete_df.csv


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
def calculer_metriques_regression(y_train, y_test, y_pred_train, y_pred_test):
    print(f"RMSE (train) :{mean_squared_error(y_train, y_pred_train, squared=False)}")
    print(f"MAE (train) :{mean_absolute_error(y_train, y_pred_train)}")
    print(f"RMSE (test) :{mean_squared_error(y_test, y_pred_test, squared=False)}")
    print(f"MAE (test) :{mean_absolute_error(y_test, y_pred_test)}")

In [ ]:
def complete_data_recuperation(file_path):
    df = pd.read_csv(file_path)
    df['date_time'] = pd.to_datetime(df['date_time'])

    df['Nombre entrees'] = df[["Payants", "Gratuits"]].sum(axis=1)

    df['jour_semaine_seance'] = df['date_time'].dt.strftime("%A")
    df['jour_seance'] = df['date_time'].dt.strftime("%d")
    df['annee_seance'] = df['date_time'].dt.strftime("%G")
    df['heure_seance'] = df['date_time'].dt.strftime("%H")
    df['mois_seance'] = df['date_time'].dt.strftime("%B")

    s = df['Genre'].apply(lambda a: literal_eval(a) if pd.notnull(a) else a).explode()
    df = df.join(pd.crosstab(s.index, s))

    try:
        df['Taux remplissage'] = df['Taux remplissage'].str.rstrip('%').str.replace(',', '.').astype('float')
    except:
        print('Nothing to do')

    df = df.replace(['N/A', -1], np.nan)

    df["Heure du lever du soleil"] = pd.to_datetime(df["Heure du lever du soleil"], format='%H:%M:%S').dt.time
    df["Heure du coucher du soleil"] = pd.to_datetime(df["Heure du coucher du soleil"], format='%H:%M:%S').dt.time
    df["Durée du jour"] = pd.to_timedelta(df["Durée du jour"])
    df["Point de rosée"] = df["Point de rosée"].str.replace('°C', '').astype('float')
    df["Température maximale"] = df["Température maximale"].str.replace('°', '').astype('float')
    df["Température minimale"] = df["Température minimale"].str.replace('°', '').astype('float')
    df["Vitesse du vent"] = df["Vitesse du vent"].str.replace('km/h', '').astype('float')
    df["Température du vent"] = df["Température du vent"].str.replace('°', '').astype('float')
    df["Précipitations"] = df["Précipitations"].str.replace('mm', '').astype('float')
    df["Humidité"] = df["Humidité"].str.replace('%', '').astype('float')
    df["Visibilité"] = df["Visibilité"].str.replace('km', '').astype('float')
    df["Couverture nuageuse"] = df["Couverture nuageuse"].str.replace('%', '').astype('float')
    df["Pression"] = df["Pression"].str.replace('hPa', '').astype('float')

    df = df.rename(columns={
        "Température maximale": "Max_temp",
        "Température minimale": "Min_temp",
        "Vitesse du vent": "Vitesse_vent",
        "Température du vent": "Temp_vent",
        "Précipitations": "Precipitations",
        "Humidité": "Humidite",
        "Visibilité": "Visibilite",
        "Point de rosée": "Point de rosee",
        "Heure du lever du soleil": "Lever_soleil",
        "Heure du coucher du soleil": "Coucher_soleil",
        "Durée du jour": "Duree du jour",
        "L'avis de  historique-meteo.net": "Avis_meteo"
    })

    cat_cols = ['Salle', 'Film', 'Version', 'Relief', 'FullTitle', 'Adult', 'Release', 'Language',
                'Lever_soleil', 'Coucher_soleil',
                'Duree du jour', 'Avis_meteo', 'nom_vacances', 'jour_semaine_seance', 'jour_seance',
                'annee_seance', 'heure_seance', 'mois_seance']

    df[cat_cols] = df[cat_cols].astype('category')

    return df

In [38]:
def search_best_params(model, param_grid, metric, k, display_train_score, X, y, n_jobs=None, **fit_params):
    """
    :param model: instance du modele 
    :param grid: grille de parametres
    :param metric: chaine de caractère précisant la métrique d'évaluation
    :param display_train_score: booleen precisant si les scores sur le dataset de train est affiché 
    :param X: features du jeu d'entrainement
    :param y: variable reponse du jeu d'entrainement
    :param fit_params (optionnel) : si fourni, déclenche l'early stopping avec les paramètres du dictionnaire
    :return: - le modele optimise avec les meilleurs parametres (objet GridSearchCV)
             - un Dataframe contenant les resultats du grid_search pour chaque combinaison de parametre
    """
    
    from time import time
    from sklearn.model_selection import GridSearchCV
    
    grid_search = GridSearchCV(model, 
                               param_grid,
                               scoring = metric,
                               cv = k, 
                               return_train_score = display_train_score, 
                               n_jobs = n_jobs
                              )
    
    tstart = time()
    grid_search.fit(X, y, **fit_params)
    tend = time() - tstart
    
    # stockage des resultats dans un DataFrame
    results_grid_search = pd.DataFrame(grid_search.cv_results_).sort_values(by = ['rank_test_score'])
    
    print("-------------- GridSearchCV {} --------------".format(model.__class__.__name__))
    mm, ss = divmod(tend, 60)
    hh, mm= divmod(mm, 60)
    print('Temps de calcul:', hh, ':', mm, ':', ss)
    print("Best parameters: {}".format(grid_search.best_params_))
    print("Best cross-validation score ({}): {:.3f} (std: {:.3f})".format(metric, grid_search.best_score_,
                    results_grid_search['std_test_score'].iloc[0]))
    print("\n")
    
    return grid_search, results_grid_search

In [ ]:
df = complete_data_recuperation(FILE_PATH)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25525 entries, 0 to 25524
Data columns (total 63 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date_time            25525 non-null  datetime64[ns]
 1   Salle                25525 non-null  category      
 2   Film                 25525 non-null  category      
 3   Version              25525 non-null  category      
 4   Relief               25525 non-null  category      
 5   Payants              23186 non-null  float64       
 6   Gratuits             3470 non-null   float64       
 7   Places libres        25495 non-null  float64       
 8   Taux remplissage     23260 non-null  float64       
 9   FullTitle            25333 non-null  category      
 10  Adult                25333 non-null  category      
 11  Release              25224 non-null  category      
 12  Language             25333 non-null  category      
 13  Genre                24840 non-

In [ ]:
# Preparation des données
df_train = df[['Salle', 'Film', 'Version', 'Relief', 'FullTitle', 'Adult',
       'Release', 'Language', 'Popularity', 'Vote_average',
       'Vote_count', 'Max_temp', 'Min_temp', 'Vitesse_vent',
       'Temp_vent', 'Precipitations', 'Humidite', 'Visibilite',
       'Couverture nuageuse', 'Indice de chaleur', 'Point de rosee',
       'Pression', 'Lever_soleil', 'Coucher_soleil', 'Duree du jour',
       'Avis_meteo', 'vacances_zone_a', 'vacances_zone_b', 'vacances_zone_c',
       'nom_vacances', 'ferie', 'Nombre entrees', 'jour_semaine_seance',
       'jour_seance', 'annee_seance', 'heure_seance', 'mois_seance', 'Action',
       'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance',
       'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']]

cat_cols = ['Salle', 'Film', 'Version', 'Relief', 'FullTitle', 'Adult', 'Release', 'Language',
            'Lever_soleil', 'Coucher_soleil',
            'Duree du jour', 'Avis_meteo', 'nom_vacances', 'jour_semaine_seance', 'jour_seance',
            'annee_seance', 'heure_seance', 'mois_seance']

df_train[cat_cols] = df_train[cat_cols].astype('category')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25525 entries, 0 to 25524
Data columns (total 56 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Salle                25525 non-null  category
 1   Film                 25525 non-null  category
 2   Version              25525 non-null  category
 3   Relief               25525 non-null  category
 4   FullTitle            25333 non-null  category
 5   Adult                25333 non-null  category
 6   Release              25224 non-null  category
 7   Language             25333 non-null  category
 8   Popularity           25333 non-null  float64 
 9   Vote_average         24024 non-null  float64 
 10  Vote_count           24024 non-null  float64 
 11  Max_temp             24187 non-null  float64 
 12  Min_temp             24187 non-null  float64 
 13  Vitesse_vent         24187 non-null  float64 
 14  Temp_vent            24187 non-null  float64 
 15  Precipitations     

In [ ]:
X_xgb = df_train.drop(columns = ['Nombre entrees'])
y_xgb = df_train['Nombre entrees']

In [ ]:
from sklearn import model_selection
X_train,X_test,y_train,y_test=model_selection.train_test_split(X_xgb,y_xgb,test_size=0.3,random_state=42)

In [19]:
# Séparation du jeu de train et d'eval
X_xgb_train,X_xgb_eval,y_xgb_train,y_xgb_eval=model_selection.train_test_split(X_train,y_train,
                                                                               test_size=0.3,
                                                                               random_state=42)
eval_set = [(X_xgb_train, y_xgb_train), (X_xgb_eval, y_xgb_eval)]

In [20]:
print(X_xgb.shape)
print('\n')
print(X_train.shape)
print(X_test.shape)
print('\n')
print(X_xgb_train.shape)
print(X_xgb_eval.shape)

(25525, 55)


(17867, 55)
(7658, 55)


(12506, 55)
(5361, 55)


In [28]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor(n_estimators=250,
                             tree_method="gpu_hist",
                             enable_categorical=True,
                             early_stopping_rounds = 20,
                             use_label_encoder=False)
model_xgb.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': 20,
 'enable_categorical': True,
 'eval_metric': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 250,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': 'gpu_hist',
 'validate_parameters': None,
 'verbosity': None,
 'use_label_encoder': False}

In [29]:
start=time()
model_xgb.fit(X_xgb_train,
              y_xgb_train,
              eval_set=eval_set,
              verbose=True)
print('temps apprentissage: '+str(time()-start))

[0]	validation_0-rmse:613.19857	validation_1-rmse:5415.60357
[1]	validation_0-rmse:2568.60099	validation_1-rmse:5680.64232
[2]	validation_0-rmse:58599.12295	validation_1-rmse:57953.30924
[3]	validation_0-rmse:34935973.43435	validation_1-rmse:35811528.29048
[4]	validation_0-rmse:5426492974.75072	validation_1-rmse:5413730378.59089
[5]	validation_0-rmse:2092539597783.13965	validation_1-rmse:1898750918274.77490
[6]	validation_0-rmse:823436403728622.37500	validation_1-rmse:1008819935776245.25000
[7]	validation_0-rmse:604029153688618112.00000	validation_1-rmse:582380980977679232.00000
[8]	validation_0-rmse:586599553476999808.00000	validation_1-rmse:566140916067146880.00000
[9]	validation_0-rmse:814389240881326080.00000	validation_1-rmse:1264154366009751808.00000
[10]	validation_0-rmse:802334791774633984.00000	validation_1-rmse:1272990138842872064.00000
[11]	validation_0-rmse:2422178401966839808.00000	validation_1-rmse:2429529520591057920.00000
[12]	validation_0-rmse:2404869544884755968.00000

In [30]:
y_xgb_pred_train = model_xgb.predict(X_train)
y_xgb_pred_test = model_xgb.predict(X_test)
calculer_metriques_regression(y_train, y_test, y_xgb_pred_train, y_xgb_pred_test)

RMSE (train) :5375.594624018611
MAE (train) :1742.299819672739
RMSE (test) :5301.9228685647795
MAE (test) :1695.9155842117143


In [53]:
# Grid search params
param_grid_xgb = {"n_estimators": [10,20],
                  "learning_rate": [0.0001, 0.001],
                  "max_depth": [3,5],
                  "early_stopping_rounds": [20]}

params_xgb_fit ={"eval_metric" : "rmse",
                 "eval_set" : eval_set,
                 "verbose":True}

model_xgb_grid = xgb.XGBRegressor(tree_method = 'gpu_hist', enable_categorical=True)


In [54]:
grid_search_xgb, results_grid_search_xgb = search_best_params(model = model_xgb_grid, 
                                                            param_grid = param_grid_xgb,
                                                            metric = "neg_root_mean_squared_error",
                                                            k = 5,
                                                            display_train_score = True,
                                                            X = X_train, 
                                                            y = y_train,
                                                            **params_xgb_fit)

[0]	validation_0-rmse:32.88354	validation_1-rmse:32.68201
[1]	validation_0-rmse:32.88312	validation_1-rmse:32.68166
[2]	validation_0-rmse:32.85274	validation_1-rmse:32.66601
[3]	validation_0-rmse:36.39553	validation_1-rmse:36.18077
[4]	validation_0-rmse:36.84745	validation_1-rmse:36.60337
[5]	validation_0-rmse:37.25741	validation_1-rmse:37.04567
[6]	validation_0-rmse:38.05786	validation_1-rmse:37.81717
[7]	validation_0-rmse:38.90639	validation_1-rmse:38.63697
[8]	validation_0-rmse:41.71261	validation_1-rmse:41.36028
[9]	validation_0-rmse:42.69465	validation_1-rmse:42.31392


/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:797: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  UserWarning,


[0]	validation_0-rmse:31.70847	validation_1-rmse:31.47936
[1]	validation_0-rmse:32.29540	validation_1-rmse:32.12145
[2]	validation_0-rmse:33.11939	validation_1-rmse:32.88844
[3]	validation_0-rmse:33.74656	validation_1-rmse:33.49403
[4]	validation_0-rmse:35.21361	validation_1-rmse:34.93374
[5]	validation_0-rmse:35.35450	validation_1-rmse:35.08601
[6]	validation_0-rmse:36.11183	validation_1-rmse:35.84336
[7]	validation_0-rmse:36.14306	validation_1-rmse:35.87131
[8]	validation_0-rmse:36.27096	validation_1-rmse:35.96817
[9]	validation_0-rmse:36.66272	validation_1-rmse:36.35454


/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:797: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  UserWarning,


[0]	validation_0-rmse:33.53437	validation_1-rmse:33.29621
[1]	validation_0-rmse:33.53246	validation_1-rmse:33.29411
[2]	validation_0-rmse:33.53048	validation_1-rmse:33.29216
[3]	validation_0-rmse:33.52856	validation_1-rmse:33.29018
[4]	validation_0-rmse:33.52672	validation_1-rmse:33.28823
[5]	validation_0-rmse:33.52482	validation_1-rmse:33.28666
[6]	validation_0-rmse:33.52319	validation_1-rmse:33.28456
[7]	validation_0-rmse:33.52146	validation_1-rmse:33.28247
[8]	validation_0-rmse:33.52075	validation_1-rmse:33.28141
[9]	validation_0-rmse:33.51893	validation_1-rmse:33.27979


...

/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:797: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  UserWarning,


[14]	validation_0-rmse:320.61216	validation_1-rmse:321.59973
[15]	validation_0-rmse:320.01240	validation_1-rmse:320.91398
[16]	validation_0-rmse:319.84198	validation_1-rmse:320.75786
[17]	validation_0-rmse:319.91128	validation_1-rmse:320.80499
[18]	validation_0-rmse:319.26974	validation_1-rmse:320.09914
[19]	validation_0-rmse:319.15806	validation_1-rmse:319.98899
[0]	validation_0-rmse:52.20246	validation_1-rmse:51.86952
[1]	validation_0-rmse:57.45679	validation_1-rmse:56.97641
[2]	validation_0-rmse:95.29643	validation_1-rmse:95.02382
[3]	validation_0-rmse:127.56505	validation_1-rmse:127.16846
[4]	validation_0-rmse:150.69658	validation_1-rmse:149.76149
[5]	validation_0-rmse:151.69494	validation_1-rmse:150.73308
[6]	validation_0-rmse:154.99014	validation_1-rmse:153.92927
[7]	validation_0-rmse:174.58091	validation_1-rmse:173.16277
[8]	validation_0-rmse:183.69344	validation_1-rmse:182.39679
[9]	validation_0-rmse:202.20422	validation_1-rmse:200.55264
[10]	validation_0-rmse:200.58437	validat

/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:797: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  UserWarning,


[15]	validation_0-rmse:229.49430	validation_1-rmse:226.80285
[16]	validation_0-rmse:233.37768	validation_1-rmse:230.85496
[17]	validation_0-rmse:236.51944	validation_1-rmse:234.13183
[18]	validation_0-rmse:239.74142	validation_1-rmse:237.52643
[19]	validation_0-rmse:241.39606	validation_1-rmse:239.28229
[0]	validation_0-rmse:34.72798	validation_1-rmse:34.47470
[1]	validation_0-rmse:34.69895	validation_1-rmse:34.44313
[2]	validation_0-rmse:34.69866	validation_1-rmse:34.44286
[3]	validation_0-rmse:34.71950	validation_1-rmse:34.45394
[4]	validation_0-rmse:35.49192	validation_1-rmse:35.22522
[5]	validation_0-rmse:36.29395	validation_1-rmse:36.02634
[6]	validation_0-rmse:37.12363	validation_1-rmse:36.85533
[7]	validation_0-rmse:37.14339	validation_1-rmse:36.87697
[8]	validation_0-rmse:38.13397	validation_1-rmse:37.86468
[9]	validation_0-rmse:38.11865	validation_1-rmse:37.85129
[10]	validation_0-rmse:40.60011	validation_1-rmse:40.36045
[11]	validation_0-rmse:41.97511	validation_1-rmse:41.747

/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:797: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  UserWarning,


In [55]:
results_grid_search_xgb[['params', 'mean_train_score', 'mean_test_score']].head(30)

,params,mean_train_score,mean_test_score
1,"{'early_stopping_rounds': 20, 'learning_rate':...",-33.281709,-33.283013
0,"{'early_stopping_rounds': 20, 'learning_rate':...",-33.282842,-33.284049
2,"{'early_stopping_rounds': 20, 'learning_rate':...",-33.528515,-33.533136
3,"{'early_stopping_rounds': 20, 'learning_rate':...",-33.528515,-33.533136
4,"{'early_stopping_rounds': 20, 'learning_rate':...",-40.182552,-40.087737
5,"{'early_stopping_rounds': 20, 'learning_rate':...",-40.182552,-40.087737
6,"{'early_stopping_rounds': 20, 'learning_rate':...",-43.936861,-43.879319
7,"{'early_stopping_rounds': 20, 'learning_rate':...",-43.936861,-43.879319


In [56]:
y_grid_xgb_pred_train = grid_search_xgb.predict(X_train)
y_grid_xgb_pred_test = grid_search_xgb.predict(X_test)
calculer_metriques_regression(y_train, y_test, y_grid_xgb_pred_train, y_grid_xgb_pred_test)

RMSE (train) :34.622107477709335
MAE (train) :21.078120838183324
RMSE (test) :35.89797596444357
MAE (test) :21.644459361381212
